In [3]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [6]:
loader1 = TextLoader("./data/nlp-keywords.txt", encoding='utf-8')
loader2 = TextLoader("./data/book_document.txt", encoding='utf-8')

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

In [8]:
# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

In [9]:
# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [10]:
# 임베딩 차원 크기를 계산
dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

1536


### **FAISS 벡터 저장소 생성 (from_documents)**

`from_documents()` 문서 리스트와 임베딩 함수를 사용하여 FAISS 벡터 저장소를 생성합니다.

**매개변수**
<br>

- documents (List[Document])  : 벡터 저장소에 추가할 문서 리스트
- embedding (Embeddings)      : 사용할 임베딩 함수
- **kwargs                    : 추가 키워드 인자

동작
<br>

1. 문서 리스트에서 텍스트 내용(page_content)과 메타데이터를 추출합니다. <br>
2. 추출한 텍스트와 메타데이터를 사용하여 from_texts 메서드를 호출합니다.

In [11]:
# DB 생성
db = FAISS.from_documents(
    documents=split_doc1+split_doc2, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

In [12]:
# 문서 저장소 ID 확인
db.index_to_docstore_id

{0: '396891ae-0fc9-4537-b096-2dd5513845e9',
 1: '48b7f169-e9ed-4d93-af52-c147d412db7e',
 2: 'a95a3e60-e225-4417-970f-d13052664ef2',
 3: '64d6efc8-5a12-4b36-be20-c649795800b8',
 4: '42ac4795-9eb6-4a01-8855-993dbd613244',
 5: '19bf47a9-28d2-44ff-be98-c4ff7a3137ec',
 6: '8f034c61-fa98-40e7-8da4-b4c56c368a28',
 7: '5035fe87-8daa-4430-9823-4e49eb2cd860',
 8: '0ef161b0-1fbf-4b46-a883-10c4d97ca5a3',
 9: 'a1d79ca6-26fa-4e0a-bdb9-82645ac44739',
 10: 'b8fbb070-ff31-4bb0-9393-2078ca8c75ce',
 11: 'd1e952de-602d-47ec-b45a-371343298ec3',
 12: '79c805a1-e629-496b-bc0c-d88386a79f90',
 13: '0e4861b0-8b20-4ca1-82ff-68ecb1edc177',
 14: 'e4d1dc56-9095-4f9a-a01a-64376218fd45'}

In [22]:
# 저장된 문서의 ID: Document 확인
# db.docstore._dict

### 유사도 검색 (Similarity Search)

**매개변수**
<br>
- query (str): 유사한 문서를 찾기 위한 검색 쿼리 텍스트
- k (int): 반환할 문서 수. 기본값은 4
- filter (Optional[Union[Callable, Dict[str, Any]]]): 메타데이터 필터링 함수 또는 딕셔너리. 기본값은 None
- fetch_k (int): 필터링 전에 가져올 문서 수. 기본값은 20
- **kwargs: 추가 키워드 인자

동작
<br>

1. similarity_search_with_score()를 내부적으로 호출하여 유사도 점수와 함께 문서를 검색합니다.
2. 검색 결과에서 점수를 제외하고 문서만 추출하여 반환합니다.

In [14]:
# 유사도 검색
db.similarity_search("TF IDF 에 대하여 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\

In [15]:
# k 값 지정
db.similarity_search("파이썬에 대한 책이 있나요?", k=2)

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 손에잡히는 파이썬\n- 저자 : 문용준\n- 발행처 : BJpublic(비제이퍼블릭) 2018\n- isbn : 9791186697726\n- 청구기호 : 일 005.133-문66ㅅ\n- 등록번호 : CLP000178588\n- 소장위치 : 세 번째 책장 두 번 째 열\n\n\n### 제목 : 작심 3일 파이썬 Python\n- 저자 : 황덕창\n- 발행처 : 스포트라잇북